In [1]:
!pip install dotenv langchain_groq langchain_core sarvamai
!pip install bert-score transformers scikit-learn

In [2]:
import requests

from dotenv import load_dotenv
import os

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
import json

from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from sarvamai import SarvamAI
import re

In [3]:
print("Current working directory:", os.getcwd())
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print('OPEN API KEY FOUND')
else:
    print("No API key found")

serp_api_key = os.getenv("SERP_API_KEY")
if serp_api_key:
    print('SERP API KEY FOUND')
else:
    print("No SERP API key found")

sarvam_api_key = os.getenv("SARVAM_API_KEY")
if sarvam_api_key:
    print('SARVAM API KEY FOUND')
else:
    print("No SARVAM API key found")

grok_api_key = os.getenv("GROK_API_KEY")
if grok_api_key:
    print('GROK API KEY FOUND')
else:
    print("No GROK API key found")

serp_dev_api_key = os.getenv("SERP_DEV_API_KEY")
if serp_dev_api_key:
    print('SERP DEV API KEY FOUND')
else:
    print("No SERP API key found")

tav_dev_api_key = os.getenv("TAVILY_API_KEY")
if tav_dev_api_key:
    print('TAV DEV API KEY FOUND')
else:
    print("No TAV API key found")

Current working directory: /content
No API key found
No SERP API key found
SARVAM API KEY FOUND
GROK API KEY FOUND
SERP DEV API KEY FOUND
No TAV API key found


In [4]:
class SerperRetrieverWrapper:
    def __init__(self, api_key: str, num_results: int = 5):
        self.api_key = api_key
        self.num_results = num_results

    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        Raises HTTPError on non-2xx responses.
        """
        _SERPER_SEARCH_URL = "https://google.serper.dev/search"

        headers = {
            "X-API-KEY": self.api_key,
            "Content-Type": "application/json"
        }
        payload = {
            "q": 'site:news18.com ' + query,
            "num": self.num_results,
        }

        resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload, timeout=5)

        if resp.status_code != 200:
            raise Exception(f"Serper API Error: {resp.text}")
        results = resp.json()

        documents = []
        for result in results.get("organic", [])[:self.num_results]:
            content = f"{result.get('title', '')}\n{result.get('snippet', '')}"
            documents.append(Document(page_content=content, metadata={"source": result.get("link", "")}))

        return documents

In [5]:
serper_retriever = SerperRetrieverWrapper(api_key=serp_dev_api_key)
context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)

In [6]:
template = """You are an AI language model assistant. Your task is to generate three
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

llm3 = ChatGroq(api_key=grok_api_key, model_name="llama3-8b-8192")
generate_queries = (
    prompt_perspectives
    | llm3
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [7]:
summarizer_prompt = PromptTemplate.from_template("""
You are an assistant summarizing factual evidence from multiple documents.

Based on the following documents, extract the key facts relevant to the claim.

Claim: {question}

Documents:
{context}

Return a short neutral summary of the key facts only.
""")

llm1 = ChatGroq(api_key=grok_api_key, model_name="llama3-8b-8192")
summarizer_chain = (
    {
        "context": context_retriever,
        "question": generate_queries   #RunnablePassthrough()
    }
    | summarizer_prompt
    | llm1
    | StrOutputParser()
)

In [8]:
judge_prompt = PromptTemplate.from_template("""
You are a fact-checking assistant.

Claim: {question}

Evidence:
{evidence}

Decide whether the claim is REAL or FAKE or UNSURE based only on the evidence.

Respond in this format:
Classification: REAL or FAKE
Explanation: <your reasoning>
""")

llm2 = ChatGroq(api_key=grok_api_key, model_name="llama3-8b-8192")
fact_checker_chain = (
    {
        "question": RunnablePassthrough(),
        "evidence": summarizer_chain
    }
    | judge_prompt
    | llm2
    | StrOutputParser()
)

In [9]:
from google.colab import files
uploaded = files.upload()

Saving claim_dataset.json to claim_dataset (2).json


In [14]:
import json
dataset = []
with open("claim_dataset (2).json") as f:
    dataset = json.load(f)
print(f"Loaded {len(dataset)} claims")

Loaded 5 claims


In [15]:
import re
def extract_label_from_llm_output(text):
    match = re.search(r"Classification:\s*(REAL|FAKE|UNSURE)", text, re.IGNORECASE)
    if match:
        return match.group(1).lower()
    return "unknown"

In [16]:
y_true_labels = []
y_pred_labels = []
candidates = []
references = []

for item in dataset:
    claim = item['claim']
    true_label = item['label'].lower()
    reference_answer = item['reference_answer']

    model_output = fact_checker_chain.invoke(claim)

    pred_label = extract_label_from_llm_output(model_output)


    y_true_labels.append(true_label)
    y_pred_labels.append(pred_label)
    candidates.append(model_output)
    references.append(reference_answer)
    print(f"{claim}\n🤖 {model_output}\n Expected: {true_label}\n")

NASA confirms Earth will go dark for 6 days!
🤖 Classification: FAKE
Explanation: The evidence provided consists of documents from the news18.com website, which do not mention or reference the claim about NASA confirming a 6-day period of darkness on Earth. The documents cover various topics such as news, sports, and politics, but there is no mention of the claim's topic. This lack of evidence supporting the claim suggests that it is likely fake.
 Expected: fake

WHO declares monkeypox a global health emergency.
🤖 Classification: REAL

Explanation: The evidence suggests that the World Health Organization (WHO) has indeed declared a global health emergency for mpox/monkeypox on at least one occasion. The text states that the first declaration was made by WHO, and the second declaration was also made by WHO, with a gap of almost a year between the two declarations. This suggests that WHO has taken formal action to declare a global health emergency in response to the surge in mpox cases an

In [17]:
from sklearn.metrics import classification_report

print("\n📊 Classification Report:\n")
print(classification_report(y_true_labels, y_pred_labels))


📊 Classification Report:

              precision    recall  f1-score   support

        fake       1.00      1.00      1.00         3
        real       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [18]:
from bert_score import score
P, R, F1 = score(candidates, references, lang="en", verbose=True)
print(f"\n🔬 Average BERTScore:\nPrecision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 9.75 seconds, 0.51 sentences/sec

🔬 Average BERTScore:
Precision: 0.8464, Recall: 0.8994, F1: 0.8721


In [15]:
claim = "Ahemdabad plane crash claimed 250 lives"
#claim = 'இஸ்ரேல் ஈரானில் அணுகுண்டு வீசுகிறது'
#claim = 'ಭಾರತವು ಎಲ್ಲಾ ವಯಸ್ಕರಿಗೆ ಸೈನಿಕ ಸೇವೆಯನ್ನು ಕಡ್ಡಾಯವಾಗಿಸಿದೆ'
#claim = 'फ़ुटबॉल विश्व कप 2030 भारत में आयोजित किया जाएगा'

In [16]:
fact_checker_chain.invoke(claim)

'Classification: REAL\n\nExplanation: The evidence provided contains specific numbers and details about the incident, including the number of fatalities (over 250) and the circumstances of the crash (crashed within a minute after takeoff). The language used is neutral and factual, without any sensationalist or contradictory statements. This suggests that the claim is based on real events and is not a fabrication.'

In [ ]:
client = SarvamAI(api_subscription_key='474af18e-b1e7-4fba-b201-cc4b4f52ed3c')

In [ ]:
try:
    translation = client.text.translate(
    input=claim,
    source_language_code="auto",
    target_language_code="en-IN"
    )
except Exception as e:
    print(f"Error during translation: {e}")

In [ ]:
#claim_final = translation.translated_text if translation else claim
print(f"Translated claim: {claim_final}")

In [ ]:
import sys
import os
def get_audio_file():
    supported_formats = ['.wav', '.mp3']
    # Running in Jupyter Notebook: input file path
    audio_file_path = input("Enter the path to your MP3 or WAV file: ").strip()
    ext = os.path.splitext(audio_file_path)[1].lower()
    if not os.path.exists(audio_file_path):
        print(f"File not found at: {audio_file_path}")
        return None
    if ext not in supported_formats:
        print(f"Unsupported file format '{ext}'. Please provide a WAV or MP3 file.")
        return None
    print(f"File '{audio_file_path}' found successfully in Jupyter!")
    return audio_file_path

'sfsd'

In [ ]:
if audio_file_path:
    with open(audio_file_path, "rb") as audio_file:
        response = client.speech_to_text.transcribe(
            file=audio_file,
            model="saarika:v2.5",
            language_code="unknown"
        )
    print("✅ Transcription Response:")
    print(response)
else:
    print("🚫 No audio file found. Transcription aborted.")

In [ ]:
try:
    translation = client.text.translate(
    input=response.transcript,
    source_language_code="auto",
    target_language_code="en-IN"
    )
except Exception as e:
    print(f"Error during translation: {e}")